In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from wordcloud import WordCloud
import matplotlib.pyplot as plt 

# 推論関数の構築
1. 類似Userの抽出
2. 類似Animeの抽出
3. おすすめするAnimeの抽出
4. おすすめされたAnimeを視聴した際の評価

In [ ]:
user = pd.read_csv("../input/anime-recommendations-database/rating.csv")
user.head()

In [ ]:
anime = pd.read_csv("../input/anime-recommendations-database/anime.csv")
anime.head()

# 集計に必要なテーブルの用意


In [ ]:

'''
ユーザーテーブルが1０００未満のユーザーを除去する。
'''

user_cnt = pd.DataFrame(user.user_id.value_counts().values, index=user.user_id.value_counts().index)
user_cnt = user_cnt.reset_index()
user_cnt.columns = ["user_id", "cnt"]
user = pd.merge(user, user_cnt, how="left", on="user_id")
user = user[user.cnt > 1000] # メモリ不足のため
user.drop("cnt", axis=1, inplace=True)
user.head()

In [ ]:
s = MinMaxScaler(feature_range=(-1.0, 1.0))
user["scaler_rating"] = s.fit_transform(user[["rating"]]).ravel()

In [ ]:
# No scaler tables 
piv = user.pivot_table(index=["user_id"], columns=["anime_id"], values=["rating"], fill_value=0.0)
piv.head()

In [ ]:
# Scaler tables 
user_anime_piv = user.pivot_table(index=["user_id"], columns=["anime_id"], values=["scaler_rating"], fill_value=0.0)
user_anime_piv.head()

In [ ]:

'''
標準化されたピボットテーブルからそれぞれの値ごとに類似度の算出をする。
* user_id x user_id 
* anime_id x anime_id 
'''

piv_sparse = sp.sparse.csr_matrix(user_anime_piv.values)
user_similar = cosine_similarity(piv_sparse)
anime_similar = cosine_similarity(piv_sparse.T)

In [ ]:
col = [int(ii) for i, ii in user_anime_piv.columns]
anime_similar = pd.DataFrame(anime_similar, 
                             columns=col, index=col)
user_similar = pd.DataFrame(user_similar, 
                             columns=user_anime_piv.index, index=user_anime_piv.index)
user_similar.head() # user_id x user_id 

In [ ]:
anime_similar.head() # anime_id x anime_id 

# 推薦するヘルパー関数

In [ ]:

'''
アニメ間の類似度
'''

def find_similar_anime(anime_name: str, n=10):
    anime_id = anime[anime.name == anime_name]["anime_id"].values[0]
    ser = anime_similar[int(anime_id)].sort_values(ascending=False)
    sim_anime_list, similar_list = ser.index[1:n+1], ser.values[1:n+1]
    df_list = []
    for sim_anime, sim in zip(sim_anime_list, similar_list):
        name = anime[anime.anime_id == sim_anime]["name"].values[0]
        genre = anime[anime.anime_id == sim_anime]["genre"].values[0]
        df_list.append({"anime_id": sim_anime, "cosine_similar": sim, "name": name, "genre": genre})
    return pd.DataFrame(df_list).sort_values("cosine_similar", ascending=False)

'''
ユーザー間の類似度
'''

def find_similar_user(user_id: int, n=10):
    ser = user_similar[int(user_id)].sort_values(ascending=False)[1:n+1]
    df_list = []
    for user_id_, sim in zip(ser.index, ser.values):
        df_list.append({"user_id": user_id_, "cosine_similar": sim})
    return pd.DataFrame(df_list).sort_values("cosine_similar", ascending=False)
    
    
def show_cloud(genre, anime):
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    ax = axes.ravel()
    genre_cloud = WordCloud().generate_from_frequencies(genre)
    anime_cloud = WordCloud().generate_from_frequencies(anime)
    ax[0].imshow(anime_cloud)
    ax[0].axis("off")
    ax[0].set_title("recommend Anime")
    ax[1].imshow(genre_cloud)
    ax[1].axis("off")
    ax[1].set_title("recommend Genres")
    fig.savefig("img.png")


'''
類似ユーザーn名から評価値が良かったトップn位の各アニメをカウントして、
ある特定のユーザーへの推薦するアニメ名とジャンルを返す関数
'''

def find_similar_user_for_anime(user_id: int, n=10):
    # 似ているユーザーIDを取得
    sim_user_id = find_similar_user(user_id, n)["user_id"]
    animeaname2count, genre2count = {}, {}
    # 似ているユーザーごとにループする
    for user_id in sim_user_id:
        # 各ユーザーの評価の高いアニメID
        for _, anime_id in user_anime_piv.loc[user_id, :].sort_values(ascending=False).index[:n]:
            # アニメ名のカウント
            anime_name = anime[anime.anime_id == anime_id]["name"].values[0]
            anime_name = anime_name.strip()
            if anime_name not in animeaname2count:
                animeaname2count[anime_name] = 1
            else:
                animeaname2count[anime_name] += 1
            # ジャンルのカウント
            genre_list = anime[anime.anime_id == anime_id]["genre"].values[0]
            for genre in genre_list.split(","):
                genre = genre.strip()
                if genre not in genre2count:
                    genre2count[genre] = 1 
                else:
                    genre2count[genre] += 1 
    show_cloud(genre2count, animeaname2count)
    
    
'''
推薦されたアニメを視聴した際に評価される値の予測
(ユーザー間の類似度*アニメ評価値) / (類似度)
つまり、ユーザー間の類似度が高いほど、評価される値も同じになると予想される。
'''

def predict_recommend_rating(user_id, n=10):
    # 上の関数化rユーザー間の類似度を取得
    sim_users = find_similar_user(user_id, n)
    sim_user_id = sim_users["user_id"]
    cosine_sim = sim_users["cosine_similar"]
    df_list = []
    # 各ユーザーが最も高い評価のアニメから、指標により評価予測する
    for user_id, cos in zip(sim_user_id, cosine_sim):
        # 類似度ユーザーによるおすすめアニメ値
        anime_id = piv.loc[user_id, :].sort_values(ascending=False).index[0][1]
        anime_rating = piv.loc[user_id, :].sort_values(ascending=False).values[0]
        anime_name = anime[anime.anime_id == anime_id]["name"].values[0]
        # 計算する
        predict_rating = (cos*anime_rating)/cos
        df_list.append({"anime_id": anime_id, "name": anime_name, "predict_rating": predict_rating})
        
    return pd.DataFrame(df_list).sort_values("predict_rating", ascending=False).drop_duplicates()[:n]


In [ ]:
random_user_id = np.random.choice(user_anime_piv.index, 3)

### Anime 
---

In [ ]:
find_similar_anime("Steins;Gate")

### User
---

In [ ]:
find_similar_user(random_user_id[0])

In [ ]:
find_similar_user(random_user_id[1])

In [ ]:
find_similar_user(random_user_id[2])

### Recommend Anime Genres
---

In [ ]:
find_similar_user_for_anime(random_user_id[0])

In [ ]:
find_similar_user_for_anime(random_user_id[1])

In [ ]:
find_similar_user_for_anime(random_user_id[2])

### Predicted Rating
---

In [ ]:
predict_recommend_rating(random_user_id[0])

In [ ]:
predict_recommend_rating(random_user_id[1])

In [ ]:
predict_recommend_rating(random_user_id[2])